In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = "{:.2f}".format

In [3]:
def get_user_matrix():
    print("Enter your rating matrix row by row.")
    print("Use space-separated numbers, and 'nan' for missing values.\n")

    # ---- validate rows ----
    while True:
        rows = int(input("Enter number of users (rows): "))
        if rows > 0:
            break
        print("Rows must be greater than 0")

    # ---- validate columns ----
    while True:
        cols = int(input("Enter number of items (columns): "))
        if cols > 0:
            break
        print("Columns must be greater than 0")

    matrix = []
    for i in range(rows):
        while True:
            row = input(f"Row {i+1}: ").split()
            if len(row) == cols:
                matrix.append([float(x) if x != 'nan' else np.nan for x in row])
                break
            print(f"Enter exactly {cols} values")

    return np.array(matrix)


# ---------- Ask User ----------
choice = input("Do you want to enter your own rating matrix? (yes/no): ").strip().lower()

if choice in ['yes', 'y']:
    R = get_user_matrix()
else:
    print("\nUsing default poster example matrix.\n")
    R = np.array([
        [5, np.nan, 4, 1, np.nan],
        [5, 3, np.nan, np.nan, np.nan],
        [np.nan, 3, 2, np.nan, 1],
        [np.nan, np.nan, np.nan, 4, 2]
    ])

R_df = pd.DataFrame(R,
                    index=["User68","User81","User145","User151"],
                    columns=["M1","M2","M3","M4","M5"])

print("Original Rating Matrix R")
R_df


Do you want to enter your own rating matrix? (yes/no):  n



Using default poster example matrix.

Original Rating Matrix R


,M1,M2,M3,M4,M5
User68,5.00,NaN,4.00,1.00,NaN
User81,5.00,3.00,NaN,NaN,NaN
User145,NaN,3.00,2.00,NaN,1.00
User151,NaN,NaN,NaN,4.00,2.00


In [4]:
R_filled = np.where(np.isnan(R), np.nanmean(R, axis=0), R)
R_filled_df = pd.DataFrame(R_filled, index=R_df.index, columns=R_df.columns)

print("Filled Rating Matrix (for decomposition)")
R_filled_df


Filled Rating Matrix (for decomposition)


,M1,M2,M3,M4,M5
User68,5.00,3.00,4.00,1.00,1.50
User81,5.00,3.00,3.00,2.50,1.50
User145,5.00,3.00,2.00,2.50,1.00
User151,5.00,3.00,3.00,4.00,2.00


In [5]:
R_T = R_filled.T
print("Transpose of R (Rᵀ)")
display(pd.DataFrame(R_T))


Transpose of R (Rᵀ)


,0,1,2,3
0,5.00,5.00,5.00,5.00
1,3.00,3.00,3.00,3.00
2,4.00,3.00,2.00,3.00
3,1.00,2.50,2.50,4.00
4,1.50,1.50,1.00,2.00


In [6]:
RtR = R_T @ R_filled
RRt = R_filled @ R_T

print("RᵀR (Item–Item space)")
display(pd.DataFrame(RtR))

print("RRᵀ (User–User space)")
display(pd.DataFrame(RRt))



RᵀR (Item–Item space)


,0,1,2,3,4
0,100.00,60.00,60.00,50.00,30.00
1,60.00,36.00,36.00,30.00,18.00
2,60.00,36.00,38.00,28.50,18.50
3,50.00,30.00,28.50,29.50,15.75
4,30.00,18.00,18.50,15.75,9.50


RRᵀ (User–User space)


,0,1,2,3
0,53.25,50.75,46.00,53.00
1,50.75,51.50,47.75,56.00
2,46.00,47.75,45.25,52.00
3,53.00,56.00,52.00,63.00


In [7]:
eig_vals_V, eig_vecs_V = np.linalg.eig(RtR)
eig_vals_U, eig_vecs_U = np.linalg.eig(RRt)

print("Eigenvectors of RᵀR → V")
display(pd.DataFrame(eig_vecs_V))

print("Eigenvectors of RRᵀ → U")
display(pd.DataFrame(eig_vecs_U))

Eigenvectors of RᵀR → V


,0,1,2,3,4
0,0.69,0.11,-0.48,-0.52,0.02
1,0.42,0.06,-0.29,0.86,-0.27
2,0.42,0.51,0.62,0.00,0.41
3,0.35,-0.85,0.26,0.00,0.28
4,0.21,-0.05,0.48,-0.01,-0.83


Eigenvectors of RRᵀ → U


,0,1,2,3
0,0.49,0.80,0.29,0.18
1,0.50,0.01,-0.87,-0.03
2,0.46,-0.20,0.29,-0.81
3,0.54,-0.56,0.29,0.55


In [8]:
singular_values = np.sqrt(np.abs(eig_vals_V))
Sigma = np.diag(singular_values)

print("Singular Values (Σ)")
display(pd.DataFrame(Sigma))



Singular Values (Σ)


,0,1,2,3,4
0,14.38,0.00,0.00,0.00,0.00
1,0.00,2.26,0.00,0.00,0.00
2,0.00,0.00,1.10,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00


In [9]:
U, sigma, Vt = np.linalg.svd(R_filled, full_matrices=False)
Sigma = np.diag(sigma)

print("Matrix U (User latent space)")
display(pd.DataFrame(U))

print("Matrix Σ (Singular values)")
display(pd.DataFrame(Sigma))

print("Matrix Vᵀ (Item latent space)")
display(pd.DataFrame(Vt))



Matrix U (User latent space)


,0,1,2,3
0,-0.49,-0.80,-0.18,-0.29
1,-0.50,-0.01,0.03,0.87
2,-0.46,0.20,0.81,-0.29
3,-0.54,0.56,-0.55,-0.29


Matrix Σ (Singular values)


,0,1,2,3
0,14.38,0.00,0.00,0.00
1,0.00,2.26,0.00,0.00
2,0.00,0.00,1.10,0.00
3,0.00,0.00,0.00,0.00


Matrix Vᵀ (Item latent space)


,0,1,2,3,4
0,-0.69,-0.42,-0.42,-0.35,-0.21
1,-0.11,-0.06,-0.51,0.85,0.05
2,0.48,0.29,-0.62,-0.26,-0.48
3,-0.50,0.72,0.20,0.14,-0.41


In [10]:
k = 2
U_k = U[:, :k]
Sigma_k = Sigma[:k, :k]
Vt_k = Vt[:k, :]

print("U_k")
display(pd.DataFrame(U_k))

print("Σ_k")
display(pd.DataFrame(Sigma_k))

print("Vᵀ_k")
display(pd.DataFrame(Vt_k))

U_k


,0,1
0,-0.49,-0.80
1,-0.50,-0.01
2,-0.46,0.20
3,-0.54,0.56


Σ_k


,0,1
0,14.38,0.00
1,0.00,2.26


Vᵀ_k


,0,1,2,3,4
0,-0.69,-0.42,-0.42,-0.35,-0.21
1,-0.11,-0.06,-0.51,0.85,0.05


In [11]:
R_reconstructed = U_k @ Sigma_k @ Vt_k

R_recon_df = pd.DataFrame(
    R_reconstructed,
    index=R_df.index,
    columns=R_df.columns
)

print("Reconstructed Rating Matrix")
R_recon_df

Reconstructed Rating Matrix


,M1,M2,M3,M4,M5
User68,5.10,3.06,3.87,0.95,1.40
User81,4.99,2.99,3.02,2.51,1.51
User145,4.57,2.74,2.56,2.73,1.43
User151,5.29,3.17,2.62,3.84,1.71


In [12]:
mask = ~np.isnan(R)
rmse = np.sqrt(np.mean((R[mask] - R_reconstructed[mask])**2))

print(f"RMSE: {rmse:.2f} approximately")

RMSE: 0.26 approximately
